In [1]:
import cv2
import numpy as np
from PersonSegmentationModule import Person_Segmentor
import time

Imported Python modules.


In [2]:
import cv2
import numpy as np
#import pycocotools.mask as mask_util

#from .blob import to_numpy
from boxes import expand_boxes


def postprocess_batch(batch_ids, scores, classes, boxes, raw_cls_masks,
                      batch_size, im_h, im_w, im_scale_y=None, im_scale_x=None, im_scale=None,
                      full_image_masks=True, encode_masks=False,
                      confidence_threshold=0.0):
    boxes_all = [np.empty((0, 4), dtype=np.float32) for _ in range(batch_size)]
    scores_all = [np.empty((0, ), dtype=np.float32) for _ in range(batch_size)]
    classes_all = [np.empty((0, ), dtype=np.float32) for _ in range(batch_size)]
    raw_masks_all = [None for _ in range(batch_size)]
    masks_all = [[] for _ in range(batch_size)]

    if batch_ids is None:
        return scores_all, classes_all, boxes_all, masks_all

    scale_x = im_scale_x
    scale_y = im_scale_y
    if im_scale is not None:
        scale_x = im_scale
        scale_y = im_scale
    assert len(scale_x) == len(scale_y)

  #  batch_ids = to_numpy(batch_ids)

    num_objs_per_batch = []
    for i in range(batch_size):
        num_objs_per_batch.append(np.count_nonzero(batch_ids == i))

    begin = 0
    for i in range(0, len(num_objs_per_batch)):
        end = begin + num_objs_per_batch[i]
        # Scale boxes back to the original image
        boxes_all[i] = boxes[begin:end]
        scores_all[i] = scores[begin:end]
        classes_all[i] = classes[begin:end]
        raw_masks_all[i] = raw_cls_masks[begin:end]
        begin = end

    # Resize segmentation masks to fit corresponding bounding boxes.
    for i in range(batch_size):
        scores_all[i], classes_all[i], boxes_all[i], masks_all[i] = \
            postprocess(scores_all[i], classes_all[i], boxes_all[i], raw_masks_all[i],
                        im_h[i], im_w[i], scale_y[i], scale_x[i], None,
                        full_image_masks, encode_masks,
                        confidence_threshold)

    return scores_all, classes_all, boxes_all, masks_all


def postprocess(scores, classes, boxes, raw_cls_masks,
                im_h, im_w, im_scale_y=None, im_scale_x=None, im_scale=None,
                full_image_masks=True, encode_masks=False,
                confidence_threshold=0.0):
    no_detections = (np.empty((0, ), dtype=np.float32), np.empty((0, ), dtype=np.float32),\
                     np.empty((0, 4), dtype=np.float32), [])
    if scores is None:
        return no_detections

    scale = im_scale
    if scale is None:
        assert (im_scale_x is not None) and (im_scale_y is not None)
        scale = [im_scale_x, im_scale_y, im_scale_x, im_scale_y]

  #  scores = to_numpy(scores)
  #1 classes = to_numpy(classes)
    #boxes = to_numpy(boxes)
  #  raw_cls_masks = to_numpy(raw_cls_masks)

    confidence_filter = scores > confidence_threshold
    scores = scores[confidence_filter]
    classes = classes[confidence_filter]
    boxes = boxes[confidence_filter]
    raw_cls_masks = list(segm for segm, is_valid in zip(raw_cls_masks, confidence_filter) if is_valid)

    if len(scores) == 0:
        return no_detections

    boxes = boxes / scale
    classes = classes.astype(np.uint32)
    masks = []
    for box, cls, raw_mask in zip(boxes, classes, raw_cls_masks):
        raw_cls_mask = raw_mask[cls, ...]
        mask = segm_postprocess(box, raw_cls_mask, im_h, im_w, full_image_masks, encode_masks)
        masks.append(mask)

    return scores, classes, boxes, masks


def segm_postprocess(box, raw_cls_mask, im_h, im_w, full_image_mask=True, encode=False):
    # Add zero border to prevent upsampling artifacts on segment borders.
    raw_cls_mask = np.pad(raw_cls_mask, ((1, 1), (1, 1)), 'constant', constant_values=0)
    extended_box = expand_boxes(box[np.newaxis, :],
                                raw_cls_mask.shape[0] / (raw_cls_mask.shape[0] - 2.0))[0]
    extended_box = extended_box.astype(int)
    w, h = np.maximum(extended_box[2:] - extended_box[:2] + 1, 1)
    x0, y0 = np.clip(extended_box[:2], a_min=0, a_max=[im_w, im_h])
    x1, y1 = np.clip(extended_box[2:] + 1, a_min=0, a_max=[im_w, im_h])

    raw_cls_mask = cv2.resize(raw_cls_mask, (w, h)) > 0.5
    mask = raw_cls_mask.astype(np.uint8)

    if full_image_mask:
        # Put an object mask in an image mask.
        im_mask = np.zeros((im_h, im_w), dtype=np.uint8)
        im_mask[y0:y1, x0:x1] = mask[(y0 - extended_box[1]):(y1 - extended_box[1]),
                                     (x0 - extended_box[0]):(x1 - extended_box[0])]
    else:
        original_box = box.astype(int)
        x0, y0 = np.clip(original_box[:2], a_min=0, a_max=[im_w, im_h])
        x1, y1 = np.clip(original_box[2:] + 1, a_min=0, a_max=[im_w, im_h])
        im_mask = np.ascontiguousarray(mask[(y0 - original_box[1]):(y1 - original_box[1]),
                                            (x0 - original_box[0]):(x1 - original_box[0])])

    if encode:
        im_mask = mask_util.encode(np.array(im_mask[:, :, np.newaxis].astype(np.uint8), order='F'))[0]
        im_mask['counts'] = im_mask['counts'].decode('utf-8')

    return im_mask

In [3]:
import cv2
import numpy as np


class Visualizer(object):

    def __init__(self, class_labels, confidence_threshold=0.5, show_boxes=False,
                 show_masks=True, show_scores=False):
        super().__init__()
        self.class_labels = class_labels
        self.confidence_threshold = confidence_threshold
        self.class_color_palette = np.asarray([2 ** 25 - 1, 2 ** 15 - 1, 2 ** 21 - 1])
        self.instance_color_palette = self.color_palette
        self.show_masks = show_masks
        self.show_boxes = show_boxes
        self.show_scores = show_scores

    def __call__(self, image, boxes, classes, scores, segms=None, ids=None):
        result = image.copy()

        # Filter out detections with low confidence.
        filter_mask = scores > self.confidence_threshold
        scores = scores[filter_mask]
        classes = classes[filter_mask]
        boxes = boxes[filter_mask]

        if self.show_masks and segms is not None:
            segms = list(segm for segm, show in zip(segms, filter_mask) if show)
            result = self.overlay_masks(result, segms, classes, ids)

        if self.show_boxes:
            result = self.overlay_boxes(result, boxes, classes)

        result = self.overlay_class_names(result, boxes, classes, scores,
                                          show_score=self.show_scores)
        return result

    def compute_colors_for_labels(self, labels):
        colors = labels[:, None] * self.class_color_palette
        colors = (colors % 255).astype(np.uint8)
        return colors

    def overlay_boxes(self, image, boxes, classes):
        colors = self.compute_colors_for_labels(classes).tolist()
        for box, color in zip(boxes, colors):
            box = box.astype(int)
            top_left, bottom_right = box[:2].tolist(), box[2:].tolist()
            image = cv2.rectangle(
                image, tuple(top_left), tuple(bottom_right), tuple(color), 1
            )
        return image

    def overlay_masks(self, image, masks, classes, ids=None):
        colors = self.compute_colors_for_labels(classes).tolist()

        segments_image = image.copy()
        aggregated_mask = np.zeros(image.shape[:2], dtype=np.uint8)
        aggregated_colored_mask = np.zeros(image.shape, dtype=np.uint8)
        black = np.zeros(3, dtype=np.uint8)

        for i, (mask, color) in enumerate(zip(masks, colors)):
            mask = mask.astype(np.uint8)
            color_idx = i if ids is None else ids[i]
            mask_color = (255,255,255)
            
            cv2.bitwise_or(aggregated_mask, mask, dst=aggregated_mask)
            cv2.bitwise_or(aggregated_colored_mask, np.asarray(mask_color, dtype=np.uint8),
                           dst=aggregated_colored_mask, mask=mask)
            
        # Fill the area occupied by all instances with a colored instances mask image.
        #cv2.bitwise_and(segments_image, black, dst=segments_image, mask=aggregated_mask)
        #cv2.bitwise_or(segments_image, aggregated_colored_mask, dst=segments_image, mask=aggregated_mask)
        # Blend original image with the one, where instances are colored.
        # As a result instances masks become transparent.
        #cv2.addWeighted(image, 0.5, segments_image, 0.5, 0, dst=image)
        #cv2.bi
     #   aggregated_colored_mask = cv2.GaussianBlur(aggregated_colored_mask,(51,51),0)
       # kernelSize = (11,11)
        #kernel = cv2.getStructuringElement(cv2.MORPH_RECT,kernelSize)
        #aggregated_colored_mask = cv2.morphologyEx(aggregated_colored_mask,cv2.MORPH_CLOSE,kernel)
        #aggregated_colored_mask = cv2.blur(aggregated_colored_mask,(51,51))
       # segments_image = 255 * segments_image/aggregated_colored_mask
        return cv2.bitwise_and(aggregated_colored_mask,segments_image), aggregated_colored_mask

    def overlay_class_names(self, image, boxes, classes, scores, show_score=True):
        labels = [self.class_labels[i] for i in classes]
        template = '{}: {:.2f}' if show_score else '{}'
        white = (255, 255, 255)

        for box, score, label in zip(boxes, scores, labels):
            s = template.format(label, score)
            textsize = cv2.getTextSize(s, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 1)[0]
            position = ((box[:2] + box[2:] - textsize) / 2).astype(int)
            cv2.putText(image, s, tuple(position), cv2.FONT_HERSHEY_SIMPLEX, .5, white, 1)

        return image

    color_palette = np.array([[0, 113, 188],
                              [216, 82, 24],
                              [236, 176, 31],
                              [125, 46, 141],
                              [118, 171, 47],
                              [76, 189, 237],
                              [161, 19, 46],
                              [76, 76, 76],
                              [153, 153, 153],
                              [255, 0, 0],
                              [255, 127, 0],
                              [190, 190, 0],
                              [0, 255, 0],
                              [0, 0, 255],
                              [170, 0, 255],
                              [84, 84, 0],
                              [84, 170, 0],
                              [84, 255, 0],
                              [170, 84, 0],
                              [170, 170, 0],
                              [170, 255, 0],
                              [255, 84, 0],
                              [255, 170, 0],
                              [255, 255, 0],
                              [0, 84, 127],
                              [0, 170, 127],
                              [0, 255, 127],
                              [84, 0, 127],
                              [84, 84, 127],
                              [84, 170, 127],
                              [84, 255, 127],
                              [170, 0, 127],
                              [170, 84, 127],
                              [170, 170, 127],
                              [170, 255, 127],
                              [255, 0, 127],
                              [255, 84, 127],
                              [255, 170, 127],
                              [255, 255, 127],
                              [0, 84, 255],
                              [0, 170, 255],
                              [0, 255, 255],
                              [84, 0, 255],
                              [84, 84, 255],
                              [84, 170, 255],
                              [84, 255, 255],
                              [170, 0, 255],
                              [170, 84, 255],
                              [170, 170, 255],
                              [170, 255, 255],
                              [255, 0, 255],
                              [255, 84, 255],
                              [255, 170, 255],
                              [42, 0, 0],
                              [84, 0, 0],
                              [127, 0, 0],
                              [170, 0, 0],
                              [212, 0, 0],
                              [255, 0, 0],
                              [0, 42, 0],
                              [0, 84, 0],
                              [0, 127, 0],
                              [0, 170, 0],
                              [0, 212, 0],
                              [0, 255, 0],
                              [0, 0, 42],
                              [0, 0, 84],
                              [0, 0, 127],
                              [0, 0, 170],
                              [0, 0, 212],
                              [0, 0, 255],
                              [0, 0, 0],
                              [36, 36, 36],
                              [72, 72, 72],
                              [109, 109, 109],
                              [145, 145, 145],
                              [182, 182, 182],
                              [218, 218, 218],
                              [255, 255, 255]], dtype=np.uint8)

In [4]:
seg = Person_Segmentor()

plugin = Person_Segmentor.init_plugin("CPU","cpu_extension_avx2.dll")

In [5]:
model_xml = r"D:\OPENVINO\person-segmentation\FP16\instance-segmentation-security-0050.xml"
model_bin = r"D:\OPENVINO\person-segmentation\FP16\instance-segmentation-security-0050.bin"

seg.load_net(model_xml,model_bin,plugin,num_requests=2)

INFO: All network layers are supported.
Input Data Shape: [1, 3, 480, 480]
Input Info Shape: [1, 3]
Output Shape: [100, 4]


In [6]:
#seg = Person_Segmentor()
plugin = Person_Segmentor.init_plugin("CPU","cpu_extension_avx2.dll")
viz = Visualizer(None)

def segment_image(img):

    img=cv2.resize(img,(400,400))
   #img = cv2.GaussianBlur(img,(3,3),0)
    result = seg.segment(img)
   
    scores, classes, boxes, masks = postprocess(result["scores"],result["classes"],result["boxes"],result["raw_masks"],400,400,im_scale=1.2)
    if (1 in classes) == False:
        return img,None,False
    else:
        classes = np.array([1])
        
  
    viz_img,mask = viz.overlay_masks(img,masks,classes)
    person = False
    if len(classes)>0 and classes[0] == 1:
        person = True
    return viz_img,mask,person

In [13]:
bg = cv2.imread(r"C:\Users\ADNEC- VW 3\Pictures\adventure_arid_background_desert_desolate_dry_dunes_600264.jpg")
bg = cv2.resize(bg,(1920,1080))

In [17]:
import numpy as np
import cv2

cap = cv2.VideoCapture(0)
viz = Visualizer(None)
cap.set(cv2.CAP_PROP_FRAME_WIDTH,1920)
cap.set(cv2.CAP_PROP_FRAME_WIDTH,1080)

while(True):
    # Capture frame-by-frame
    ret, frame = cap.read()
    if ret == False:
        break
    person = False
    # Our operations on the frame come here
    try:
        frame,mask,person = segment_image(frame)
    except:
        pass
    
    # Display the resulting frame
    if person:
        #foreground = cv2.resize(frame,(800,600))
        #background = cv2.bitwise_or(cv2.resize(mask,(800,600)),bg)
        final = cv2.bitwise_and(frame,mask)
        final = cv2.resize(final,(1920,1080))
        mask = cv2.resize(mask,(1920,1080))
        #foreground = cv2.bitwise_or(bg,mask)
        background = bg.copy()
        background[mask == 255] = 0
        #rgb_bg_masked[rgb_hand_mask == 0] = [0,0,0]
       # final = cv2.addWeighted(final,0.7,cv2.GaussianBlur(final,(51,51),0),-0.1,0)
        #cv2.addWeighted(final, 0.9, background, 0.5, 0, dst=final)
        final_blurred = cv2.GaussianBlur(final,(51,51),0)
        final = cv2.addWeighted(final,0.8,final_blurred,0.2,0)
        cv2.imshow('frame',cv2.add(final ,background))
    else:
        cv2.imshow('frame',cv2.resize(frame,(1920,1080)))
                    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# When everything done, release the capture
cap.release()
cv2.destroyAllWindows()

In [28]:
m = np.ones((3,3))

m[:,0:2] = 3

m

array([[3., 3., 1.],
       [3., 3., 1.],
       [3., 3., 1.]])

In [12]:
cap.release()
cv2.destroyAllWindows()

In [91]:
labels = ["Person"]
for i in range(1,100):
    labels.append("N/A")

viz = Visualizer(labels,show_scores= True)
im = viz.overlay_boxes(img,result["boxes"],result["classes"])

cv2.imshow("Image",im)
cv2.waitKey(0)
cv2.destroyAllWindows()

NameError: name 'img' is not defined

In [50]:
result["classes"]

array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
      dtype=float32)

In [36]:
(min_x,min_y,max_x,max_y) = result["boxes"][0]

cv2.rectangle(img,(min_x,min_y),(max_x,max_y),(255,255,0),3)

cv2.imshow("Image",img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [33]:
result["raw_masks"].shape

(100, 81, 14, 14)

In [18]:
masked = np.zeros((400,400),dtype = "uint8")
cv2.drawContours(masked,result["raw_masks"][0][0],-1,(255,255,255),-1)

NameError: name 'result' is not defined

In [43]:
result["classes"]

array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
      dtype=float32)